<a href="https://colab.research.google.com/github/vin3697/Face_detection_with_FaceNet-Haarcascade/blob/main/facedetection_with_FaceNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os import listdir
from PIL import Image as Img
from numpy import asarray
from numpy import expand_dims
from keras.models import load_model
import numpy as np
import tensorflow as tf

import pickle
import cv2

In [ ]:
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
MyFaceNet = load_model("/content/face_detection/workspace/model_files/facenet_keras.h5") 

In [ ]:
my_folder='/content/face_detection/workspace/picture_sample/'
database = {}

for filename in listdir(my_folder):

    path = my_folder + filename
    gbr_image_old = cv2.imread(my_folder + filename)
    
    image = HaarCascade.detectMultiScale(gbr_image_old,1.1,4) # 1.1 is scaleFactor – Parameter specifying how much the image size is reduced at each image scale.
    #4 is minNeighbors – Parameter specifying how many neighbors each candidate rectangle should have to retain it.
    
    if len(image)>0:
        x1, y1, width, height = image[0]         
    else:
        x1, y1, width, height = 1, 1, 10, 10
        
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    gbr = cv2.cvtColor(gbr_image_old, cv2.COLOR_BGR2RGB)
    gbr = Img.fromarray(gbr)                  
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Img.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)
    
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = MyFaceNet.predict(face)
    
    database[os.path.splitext(filename)[0]]=signature

    with open('/content/face_detection/workspace/database', 'w') as writefile:
      writefile.write(str(signature))

In [ ]:
myfile = open("/content/face_detection/workspace/data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()

In [ ]:
myfile = open("/content/face_detection/workspace/data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [ ]:
cap = cv2.VideoCapture(0)

while(1):
    dump, gbr_img = cap.read()
    
    image = HaarCascade.detectMultiScale(gbr_img,1.1,4)
    
    if len(image)>0:
        x1, y1, width, height = image[0]        
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    
    gbr = cv2.cvtColor(gbr_img, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)
    
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = MyFaceNet.predict(face)
    
    min_dist=100
    identity=' '
    for key, value in database.items() :
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            
    cv2.putText(gbr_img,identity, (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr_img,(x1,y1),(x2,y2), (0,255,0), 2)
        
    cv2.imshow('Face_detected_of_mine',gbr_img)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

error: ignored